# 🤓 NerdBud – Interactive Learning Dashboard

An AI-powered personalized learning companion for programming education.


In [74]:
import os
import json
import time
import joblib
import random
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display


In [75]:
BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))

MODEL_PATH = os.path.join(BASE_PATH, "models", "nerdbud_model.pkl")
QUIZ_PATH = os.path.join(BASE_PATH, "data", "quizzes", "python_basics.json")

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError("Run 06_ModelTraining.ipynb first to create the model.")

model = joblib.load(MODEL_PATH)


In [76]:
with open(QUIZ_PATH, "r") as f:
    quiz_data = json.load(f)


In [77]:
username = widgets.Text(
    description="Name:",
    placeholder="Enter your name"
)

level = widgets.Dropdown(
    options=["Beginner", "Intermediate", "Advance"],
    description="Level:"
)

start_nerdbud_btn = widgets.Button(
    description="Start NerdBud",
    button_style="success"
)

start_quiz_btn = widgets.Button(
    description="Start Quiz",
    button_style="info"
)

output = widgets.Output()


In [78]:
current_q = 0
responses = []
question_start_time = None


In [79]:
def rule_based_decision(data):
    if data["accuracy"] >= 0.8 and data["avg_time"] <= 20:
        return "Advance"
    return "Revise"


In [80]:
def ml_based_decision(data):
    df = pd.DataFrame([data])
    pred = model.predict(df)[0]
    return "Advance" if pred == 1 else "Revise"


In [81]:
def render_question(level):
    global question_start_time

    output.clear_output()
    questions=quiz_data[level.value]
    q = questions[current_q]

    q_label = widgets.HTML(f"<h4>{q['question']}</h4>")
    
    shuffled_options = q["options"].copy()
    random.shuffle(shuffled_options)

    options = widgets.RadioButtons(options=shuffled_options)

    submit_btn = widgets.Button(description="Submit")

    question_start_time = time.time()

    def submit_answer(b):
        global current_q

        time_taken = round(time.time() - question_start_time, 2)
        correct = options.value == q["answer"]

        responses.append({
            "topic": q["topic"],
            "correct": int(correct),
            "time_taken": time_taken
        })

        current_q += 1

        if current_q < len(questions):
            render_question(level)
        else:
            show_results()

    submit_btn.on_click(submit_answer)

    with output:
        display(q_label, options, submit_btn)


In [82]:
def compute_metrics():
    df = pd.DataFrame(responses)

    accuracy = df["correct"].mean()
    avg_time = df["time_taken"].mean()
    topic_perf = df.groupby("topic")["correct"].mean()

    return accuracy, avg_time, topic_perf, df


In [83]:
def plot_charts(df, topic_perf):
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    topic_perf.plot(kind="bar", ax=axes[0], title="Accuracy per Topic")
    axes[0].set_ylim(0, 1)

    df["time_taken"].plot(kind="line", ax=axes[1], title="Time per Question")
    axes[1].set_ylabel("Seconds")

    plt.show()


In [84]:
def show_results():
    output.clear_output()

    accuracy, avg_time, topic_perf, df = compute_metrics()

    ml_input = {
        "accuracy": round(accuracy, 2),
        "avg_time": round(avg_time, 2),
        "attempts": len(df)
    }

    rule_decision = rule_based_decision(ml_input)
    ml_decision = ml_based_decision(ml_input)

    with output:
        display(f"📊 Accuracy: {accuracy:.2f}")
        display(f"⏱ Avg Time: {avg_time:.2f} sec")
        display(f"📌 Rule-Based Decision: {rule_decision}")
        display(f"🤖 ML-Based Decision: {ml_decision}")

        display("### 📈 Topic Performance")
        display(topic_perf)

        plot_charts(df, topic_perf)

        if ml_decision == "Advance":
            display("🚀 NerdBud: You are ready to advance!")
        else:
            display("📘 NerdBud: Revise weak topics before advancing.")


In [85]:
def start_nerdbud(b):
    output.clear_output()
    with output:
        display(f"👋 Welcome {username.value}")
        display("Click below to start your quiz.")
        display(start_quiz_btn)

def start_quiz(b):
    global current_q, responses
    current_q = 0
    responses = []
    render_question(level)

start_nerdbud_btn.on_click(start_nerdbud)
start_quiz_btn.on_click(start_quiz)


In [86]:
display(username, level, start_nerdbud_btn, output)

Text(value='', description='Name:', placeholder='Enter your name')

Dropdown(description='Level:', options=('Beginner', 'Intermediate', 'Advance'), value='Beginner')

Button(button_style='success', description='Start NerdBud', style=ButtonStyle())

Output()